In [1]:
import requests
import pytz
import time
import json
from datetime import datetime, timedelta
from kafka import KafkaProducer

In [3]:
topicName ='Speedband'
bootstrap_servers = ['localhost:9092']

# Singapore time zone
sgt = pytz.timezone('Asia/Singapore')

# URL and headers
url = "http://datamall2.mytransport.sg/ltaodataservice/v3/TrafficSpeedBands"
headers = {
  'AccountKey': 'TdT4Q/DhRjC9n4ZkUHrYww=='
}

# Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

def fetch_data(skip):
    response = requests.get(f"{url}?$skip={skip}", headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

def fetch_real_time_data():
    all_data = []
    skip = 0
    while True:
        data = fetch_data(skip)
        if data:
            all_data.extend(data['value'])
            if len(data['value']) < 500:
                break
            else:
                skip += 500
        else:
            break
    return all_data

In [4]:
while True:
    current_time = datetime.now(sgt)
    next_hour = (current_time + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    
    print("当前时间：", current_time)
    
    all_data = fetch_real_time_data()

    if all_data:
        all_data_with_datehour = []
        for item in all_data:
            item['datehour'] = current_time.strftime("%Y-%m-%d %H")
            all_data_with_datehour.append(item)

        producer.send(topicName, all_data_with_datehour)
        print("Message sent")
        producer.flush()

    # Calculate sleep time until next whole hour
    sleep_time = (next_hour - datetime.now(sgt)).total_seconds()

    # Sleep until next whole hour
    time.sleep(sleep_time)

Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent
Message sent


KeyboardInterrupt: 